In [1]:
import os
import numpy as np
import pandas as pd
import p2pspatial

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-03-29 19:11:19,715 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
class ValidAxonMapModel(p2pspatial.models.SRDLossMixin, p2pspatial.models.RetinalGridMixin,
                        p2pspatial.models.AxonMapMixin):
    """A class that implements all abstract methods of AxonMapModel"""

    def build_optic_fiber_layer(self):
        self.axon_contrib = [np.zeros((1, 3))] * int(np.prod(self.xret.shape))

    def _predicts_target_values(self, row):
        return row

    def _predicts(self, Xrow):
        """Returns the input (a DataFrame row, without its index)"""
        _, row = Xrow
        return self._predicts_target_values(row)

In [3]:
subject = '12-005'
amplitude = 2.0
electrodes = None
random_state = 42

In [4]:
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')
X, y = p2pspatial.load_data(rootfolder, subject=subject, electrodes=electrodes,
                            amplitude=amplitude, random_state=random_state,
                            single_stim=True, verbose=False)
X = X[:10]
y = y[:10]

In [5]:
model = p2pspatial.models.ModelC(engine='serial', xystep=1)
model

ModelC(ax_segments_range=(3, 50), axlambda=100, axons_range=(-180, 180),
    engine='serial', greater_is_better=False, img_thresh=0.1,
    implant_rot=0, implant_type=<class 'pulse2percept.implants.ArgusII'>,
    implant_x=0, implant_y=0, loc_od_x=15.5, loc_od_y=1.5,
    n_ax_segments=301, n_axons=301, n_jobs=-1, rho=100,
    scheduler='threading', w_dice=34, w_rot=33, w_scale=34,
    xrange=(-30, 30), xystep=1, yrange=(-20, 20))

In [6]:
model.build_ganglion_cell_layer()

In [7]:
bundles = model._grows_axon_bundles()

In [8]:
axons = model._finds_closest_axons(np.array(bundles))

In [9]:
axon_contrib = model._calcs_axon_contribution(axons)

In [10]:
%timeit model._grows_axon_bundles()

46.7 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit model._finds_closest_axons(bundles)

580 ms ± 5.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
model.set_params(engine='cython')
model._finds_closest_axons(bundles)

199 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
model.set_params(engine='joblib')
model._calcs_axon_contribution(axons)

138 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
model.set_params(engine='cython')
model._calcs_axon_contribution(axons)

23.8 ms ± 270 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
model.set_params(engine='cython')
ax1 = model._finds_closest_axons(bundles)
model.set_params(engine='joblib')
ax2 = model._finds_closest_axons(bundles)
np.allclose([np.allclose(ax1[a], ax2[a]) for a in range(len(ax1))], True)

True

In [25]:
model.set_params(engine='cython')
ax1 = model._calcs_axon_contribution(axons)
model.set_params(engine='joblib')
ax2 = model._calcs_axon_contribution(axons)
for a1, a2 in zip(ax1, ax2):
    if len(a1) != len(a2):
        print('wrong length')
        continue
    if not np.allclose(a1, a2):
        print('not close')

wrong length
wrong length
wrong length
wrong length
not close
not close
not close
not close
not close
not close
wrong length
wrong length
wrong length
not close
not close
not close
not close
not close
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
wrong length
not close
wrong length
wrong length
wrong length
not close
not close
wrong length


In [ ]:
ax1[0]

In [ ]:
ax2[0]